<img src="logo2.png"></img>

# Week 4: Final Challenge
**Welcome to the final challenge!**<br/>
Congratulations for coming thus far. In the final challenge, you will get to utilize what you learned in the past three weeks. <br/>

As this final challenge will serve as the basis for the actual competition, please note that the organizers will not be able to assist you in getting the right answers. Please limit questions only pertaining to clarification of rules or guidelines. <br/>
Thank you and good luck with the challenge! 

***************************
# The Final Challenge Problem

Zed City is a newly established (fictitious) municipality in Tokyo and is made up of 11 districts. <br/> 
Four convenience store ( _konbini_ ) chains A, B, C and D have each established their first store in this new city in non-overlapping districts.
The map (or graph) below shows the 11 districts of Zed City and which district has a konbini already.

Since this map looks different from a conventional one, let us explain how you should look at it once more just in case.<br/>
If you count the number of nodes in this map (or graph), you'll notice that there are 11 of them. So, you should be able to tell that each node in this map (or graph) represents one of the 11 districts of Zed city. The colored nodes are the districts that have konbinis already with each color representing a different konbini chain. In this graph, konbini chain A is represented in Red, B in Blue, C in Yellow and D in Green. Next, you should take notice of the edges that connect these nodes. Any node (district) connected to each other by an edge means that they are districts adjacent to each other.
<img src="./tokyo_map_pic.png" width="700">
As the mayor of Zed City, you want to establish konbinis in the rest of the districts that still don't have one yet.<br/>
Upon your request, all four konbini chains discussed with each other and agreed to establish their konbinis in Zed City under the following two conditions:

**-Only one konbini is allowed in one district.**<br/>
**-No two adjacent districts can have konbinis from the same chain.**<br/>

Can you come up with a plan that satisfies the above conditions? You must provide all store plan combinations that meet the conditions above.<br/>

**For solving this problem:**<br/>
- You must use the 32-qubit simulator as your backend. (i.e. backend = provider.get_backend('ibmq_qasm_simulator'))
- Use **Grover's algorithm** you learned in Week2 & 3 with **iteration ＝ 5.**
- Each konbini chain A, B, C and D should be described (mapped) as **A: $00$、B:$01$、C:$10$、D:$11$**.
- A konbini you are establishing in **district _n_**, should be mapped into classical registers **c[2n]** and **c[2n+1]**.
(For example, let's say you want to establish konbini chain B:$01$ into district n=1. In this case, you should map $0$ and $1$ into classical registers c[2] and c[3].)
- Make sure you **create an oracle** that **doesn't require any knowledge of what the answers are**. (For example, you are not allowed to create an oracle by using a classical optimization solver to get your answers for it.)  
- With the exception of the Unroller, which is required for decomposing your circuit to calculate quantum costs, you are not allowed to use any existing transpiler passes nor original transpilers for making simple simplifications in this competition.
- You are not allowed to use the reset operation (i.e. qc.reset()) in your circuit for this competition.
<br/><br/>
***************************

# Answer Program

In [1]:
# Useful additional packages 
# import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.aqua.circuits.gates import mct

## Assigning qubits
<pre>
Qubit assignments:
q[0]-q[13]: districts
q[14]: oracle qubit
q[15]-q[30]: judges
q[31]: ancilla
</pre>

In [2]:
q = QuantumRegister(32)
c = ClassicalRegister(14)
qc = QuantumCircuit(q, c)

districts = [q[x] for x in range(14)]
oracle_qubit = q[14]
judges = [q[x] for x in range(15, 31)]
ancilla_qubit = q[31]

## Check if district i == district j

In [3]:
def check_district(tupl, judge):
    i = tupl[0]
    j = tupl[1]
    qc.x(q[2*j])
    qc.x(q[2*j+1])
    qc.cx(q[2*i], q[2*j])
    qc.cx(q[2*i+1], q[2*j+1])
    qc.x(judge)
    qc.ccx(q[2*j], q[2*j+1], judge)
    qc.cx(q[2*i], q[2*j])
    qc.cx(q[2*i+1], q[2*j+1])
    qc.x(q[2*j])
    qc.x(q[2*j+1])

## Check boundary condition

In [4]:
A_neighbor = [0, 2, 3]
B_neighbor = [1, 4]
C_neighbor = [2]
D_neighbor = [5, 6]

def check_not_D(district, judge):
    qc.ccx(q[2*district], q[2*district+1], judge)
    qc.x(judge)

def check_not_A(district, judge):
    qc.x(q[2*district])
    qc.x(q[2*district+1])
    check_not_D(district, judge)
    qc.x(q[2*district])
    qc.x(q[2*district+1])

def check_not_B(district, judge):
    qc.x(q[2*district])
    check_not_D(district, judge)
    qc.x(q[2*district])

def check_not_C(district, judge):
    qc.x(q[2*district+1])
    check_not_D(district, judge)
    qc.x(q[2*district+1])

def check_bc(js):
    if len(js) < 8:
        raise AquaError('Not enough judge qubits')
    else:
        i = 0
        for district in A_neighbor:
            check_not_A(district, js[i])
            i += 1
        for district in B_neighbor:
            check_not_B(district, js[i])
            i += 1
        for district in C_neighbor:
            check_not_C(district, js[i])
            i += 1
        for district in D_neighbor:
            check_not_D(district, js[i])
            i += 1

## Nice utility function

In [5]:
def and_op(q0, q1, target):
    qc.ccx(q0, q1, target)

## Oracle

In [6]:
edges = [(0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 3), \
         (2, 5), (2, 6), (3, 4), (3, 5), (3, 6), (4, 6), \
         (5, 6)]

def oracle():
    check_district(edges[-1], judges[8])
    check_district(edges[-2], judges[9])
    check_bc(judges[:8])
    for i in range(5):
        and_op(judges[2*i], judges[2*i+1], judges[i+11])
    check_bc(judges[:8])
    check_district(edges[-1], judges[8])
    check_district(edges[-2], judges[9])

    for i in range(11):
        check_district(edges[i], judges[i])

    ancilla = districts + [ancilla_qubit]
    qc.mct(judges, oracle_qubit, ancilla, mode='basic-dirty-ancilla')

def clean_judges_after_oracle():
    for i in range(11):
        check_district(edges[i], judges[i])

    check_district(edges[-1], judges[8])
    check_district(edges[-2], judges[9])
    check_bc(judges[:8])
    for i in range(5):
        and_op(judges[2*i], judges[2*i+1], judges[i+11])
    check_bc(judges[:8])
    check_district(edges[-1], judges[8])
    check_district(edges[-2], judges[9])

## Initialize district qubits

In [7]:
def initialize_districts():
    for i in range(14):
        qc.h(districts[i])

## Diffusion

In [8]:
def diffusion(mode='basic'):
    initialize_districts()
    for qubit in districts:
        qc.x(qubit)
    qc.h(districts[-1])

    ancilla = judges + [ancilla_qubit]
    qc.mct(districts[:-1], districts[-1], ancilla, mode=mode)

    qc.h(districts[-1])
    for qubit in districts:
        qc.x(qubit)
    initialize_districts()

## The rest of the program: Grover's algorithm

In [9]:
iteration = 5

initialize_districts()
qc.x(oracle_qubit)
qc.h(oracle_qubit)
for i in range(iteration-1):
    oracle()
    clean_judges_after_oracle()
    diffusion('basic')
oracle()
diffusion('basic-dirty-ancilla')
qc.barrier()
qc.measure(districts, c)

## Unroller

In [10]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(qc) 
# new_circuit.draw(output='mpl')
new_circuit.count_ops()

OrderedDict([('u3', 6861), ('cx', 3756), ('measure', 14), ('barrier', 1)])

#  IMPORTANT
- Depending on your circuit size, **execution time can get quite long**. Sometimes even up to 60 minutes. So please be patient. 
- Please **do not throw jobs in succession** even if you are worried whether your job is running properly or not. This can create a long queue and clog the backend. You can check if your job is properly running from:<br/>
https://quantum-computing.ibm.com/results  
- **For the final challenge submission**, please make sure to **use the following code for running your circuit** on the 32-qubit QASM simulator. 

In [11]:
from qiskit import IBMQ
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count = result.get_counts()
print(count)
#`shots` are set to 8000 to increase sampling
#`seed_simulator`` is set to 12345 to 'lock' its value, and 
#`backend_options={"fusion_enable":True}` is specified to improve simulator performance.

{'01010001001111': 2, '01001101010110': 1, '00111110111110': 3, '10110111000100': 1, '01010110011111': 1, '00001101101111': 1, '10101001010100': 1, '00100101111001': 1, '00011001100101': 2, '00010011001010': 1, '11000010010000': 1, '01000111010010': 1, '10011000101110': 1, '00011001011000': 1, '11011001010000': 1, '11100101011000': 1, '10011001001011': 2, '01101111101101': 2, '00000010101100': 1, '10110110010101': 1, '10100110010101': 1, '01110111101001': 1, '01110000110111': 2, '00001100000101': 1, '01010101010011': 1, '10000110100001': 1, '01001101001110': 1, '10001010100100': 1, '01001111001010': 1, '00001000101000': 2, '01000100101010': 1, '01110100001010': 1, '00011000110110': 1, '10010010100001': 1, '10010101111001': 2, '11110111011001': 1, '10010111001011': 2, '11010000111110': 1, '11111111000001': 1, '11010111011101': 3, '01011010001011': 2, '00110001110011': 1, '10110001000000': 1, '01011000101010': 1, '10000110100111': 2, '01100100010110': 1, '11110001010001': 1, '10010100001

#  SUBMISSION GUIDELINES
### Expected output:
While making sure you adhere to all the rules, please obtain the following results:<br/>
(1)Figure out the possible store plans by extracting the top nine bit-strings that give you the highest probabilities.<br/>
(2)Find out the elementary gate (u3 and CX) counts by applying the Unroller to the circuit you built.

### For Submission:
#### Use the following program to create your submission file
Using the **Submission File Creation Program** below, input your circuit, its result, your team name and the number of times you have made your submissions each into `circuit`, `results`, `name` and `times`. This will create a single submission file that will provide the above (1) and (2) in one file. 
Please note that the generated submission file will show konbini chains A, B, C and D described as 0, 1, 2, 3.

#### You need to submit:
#### File 1. The ouput file created by the file creation program (.txt file)
#### File 2. The program of the circuit you built (.py file)

#### Submissions that do not comply with the above format will be rejected. 
If you want to check if your submission file is compliant with the specified format, you can always verify this by using the **Verification Program** provided in the last section of this page.

#### Example Output for  File 1
The Submission File Creation Program will give you an output text file that will look something like: <br/>
(Note: These are not the actual answers! Just a sample to show you how your output should look like.) 

```
{"ans": [["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], ["01230123012", 30], "costs": {"u3": 20000, "cx": 10000, "barrier": 20, "measure": 20}}
```
**Use this [Submission Form](https://angelhack.typeform.com/to/MpHd9o) to make submissions for the Final Challenge.**

## Submission File Creation Program

In [12]:
# Input your quantum circuit
circuit = qc
# Input your result of the execute(groverCircuit, backend=backend, shots=shots).result()
results = result
count=results.get_counts()
# Provide your team name
name='Input your team name as exactly as you inputted it in the team details form'
# Please indicate the number of times you have made a submission so far. 
# For example, if it's your 1st time to submit your answer, write 1. If it's your 5th time to submit your answer, write 5.
times='Input the number of times you have made your submissions so far'

In [13]:
import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
# pass_ = Unroller(['u3', 'cx'])
# pm = PassManager(pass_)
# new_circuit = pm.run(circuit) 

# obtain gates
gates=new_circuit.count_ops()

#sort count
count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)

# collect answers with Top 9 probability
ans_list = count_sorted[0:9]

# reverse ans_list
ans_reversed = []
for i in ans_list:
    ans_temp=[i[0][::-1],i[1]]
    ans_reversed.append(ans_temp)

# convert each 2 bits into corresponding color. Add node0(0),node3(1),node8(2) and node11(3)
ans_shaped = []
for j in ans_reversed:
    ans_temp=j[0]
    nodeA = 0
    node0 = int(ans_temp[0] + ans_temp[1], 2)
    node1 = int(ans_temp[2] + ans_temp[3], 2)
    nodeB = 1
    node2 = int(ans_temp[4] + ans_temp[5], 2)
    node3 = int(ans_temp[6] + ans_temp[7], 2)
    node4 = int(ans_temp[8] + ans_temp[9], 2)
    nodeC = 2
    node5 = int(ans_temp[10] + ans_temp[11], 2)
    node6 = int(ans_temp[12] + ans_temp[13], 2)
    nodeD = 3
    nodes_color = str(nodeA) + str(node0) + str(node1) + str(nodeB) + str(node2) + str(node3) + str(node4) + str(nodeC) + str(node5) + str(node6) + str(nodeD) 
    ans_shaped.append([nodes_color,j[1]])

# write the result into '[your name]_final_output.txt'
# filename=name+'_'+times+'_final_output.txt'
dct={'ans':ans_shaped,'costs':gates}
# with open(filename, 'w') as f:
#     json.dump(dct, f)
print(dct)

{'ans': [['02013132023', 56], ['02313102023', 46], ['01013232103', 39], ['02011322203', 38], ['01313202013', 37], ['02013122203', 33], ['01013232013', 33], ['02313122203', 33], ['02013132203', 32]], 'costs': OrderedDict([('u3', 6861), ('cx', 3756), ('measure', 14), ('barrier', 1)])}


In [14]:
success_count = 0
for a in dct['ans']:
    d = a[0]
    if (d[0]!=d[1]) & (d[0]!=d[4]) & (d[0]!=d[5]) & (d[1]!=d[2]) & \
        (d[1]!=d[4]) & (d[1]!=d[5]) & (d[2]!=d[3]) & (d[2]!=d[5]) & \
        (d[2]!=d[6]) & (d[3]!=d[6]) & (d[4]!=d[5]) & (d[4]!=d[7]) & \
        (d[4]!=d[8]) & (d[4]!=d[9]) & (d[5]!=d[6]) & (d[5]!=d[8]) & \
        (d[5]!=d[9]) & (d[6]!=d[9]) & (d[8]!=d[9]) & (d[8]!=d[10]) & \
        (d[9]!=d[10]):
        print('Success!')
        success_count += 1
    else:
        print('This is a failure, youre a failure')
if success_count >= 9:
    print('Mayor of Zed City is happy!')

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Mayor of Zed City is happy!


## Verification Program

In [ ]:
# Input the path of your submission file
your_path='./output/bob_1_final_output.txt'

In [ ]:
import json
from pathlib import Path

p= Path(your_path)

# Verify your information
f_name=p.name
your_info=f_name.split('_')
print('Your name: ', your_info[0])
print('The number of times you have submitted your answer: ', your_info[1])

with open(p, 'r') as f:
    print(f)
    your_ans=json.load(f)

print('Does your submission file have 9 answers?')
if (len(your_ans['ans'])!=9):
    print('- No, make sure you have 9 answers with top 9 probabilities')
else:
    print('- Yes')
    print('- Your plan: ', your_ans['ans'])

print('What is your cost?')
your_cost=your_ans['costs']['u3'] + 10*your_ans['costs']['cx']